pip install -q git+https://github.com/huggingface/trl

In [11]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# The instruction dataset to use
dataset_name = "Conjuror64/C64-inst-format"

# Fine-tuned model name
new_model = "Conjuror64/mistral-C64Wizard-PEFT"


hub_name = "pechaut/Mistral-C64Wizard-instruct"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"



max_steps = 1000 # to tweak to get the best out of the model 
nb_epochs = 3

In [2]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="C64Wizard",
    
    # track hyperparameters and run metadata
    config={

    "epochs":nb_epochs,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pierre-emmanuel-chaut. Use `wandb login --relogin` to force relogin


In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import os
 
# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, add_eos_token=True, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 18610

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             quantization_config=bnb_config,
                                             low_cpu_mem_usage=True,
                                             device_map="auto"
                                             )

c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\transformers\models\auto\tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.37s/it]


In [4]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import os

model.resize_token_embeddings(len(tokenizer))
model.config.use_cache=False
model.config.pretraining_tp=1
model.config.window = 512 
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


Loading Dataset

In [5]:
from datasets import load_dataset


# Load the dataset
dataset = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
split_dataset = dataset.train_test_split(test_size=0.001)
test_dataset = split_dataset["test"]
train_dataset = split_dataset["train"]

c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\datasets\load.py:2555: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Generating train split: 100%|██████████| 8798/8798 [00:00<00:00, 324985.79 examples/s]


In [6]:



peft_config = LoraConfig(
    r=64, lora_alpha=16, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM", target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",

    ],inference_mode = False
)



model = get_peft_model(model, peft_config)
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing = True,

    evaluation_strategy="steps",
    learning_rate=2e-5,
    lr_scheduler_type="constant",
    warmup_ratio=0.03,
    max_grad_norm=0.3,
    save_strategy="epoch",
    logging_dir="./logs", 
    logging_steps=50,
    num_train_epochs=nb_epochs,
    group_by_length=True,
    fp16=False,
    report_to="wandb",
    push_to_hub=True,
    adam_beta2=0.999,
         do_train=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=None,
    neftune_noise_alpha=5
)


c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\trl\trainer\sft_trainer.py:257: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(
Map: 100%|██████████| 9/9 [00:00<00:00, 598.58 examples/s]


In [ ]:

trainer.train()
trainer.model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

---
If you want to merge and push now.. done on CPU

---

In [5]:
from huggingface_hub import login, logout
login()

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import os
new_model = "Conjuror64/mistral-C64Wizard-PEFT"
tokenizer = AutoTokenizer.from_pretrained(new_model, use_fast=True)
new_model= "pechaut/Mistral-C64Wizard-instruct-PEFT"

tokenizer.push_to_hub(new_model)


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pechaut/Mistral-C64Wizard-instruct-PEFT/commit/6befc980c8675c43c88361a622c72d7e468ff3fd', commit_message='Upload tokenizer', commit_description='', oid='6befc980c8675c43c88361a622c72d7e468ff3fd', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

new_model = "Conjuror64/mistral-C64Wizard-PEFT"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=new_model,
                                             trust_remote_code=True,
                                             )
new_model= "pechaut/Mistral-C64Wizard-instruct-PEFT"
model.push_to_hub(new_model,max_shard_size="1GB")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pe_ch\.cache\huggingface\hub\models--pechaut--Mistral-C64Wizard-instruct-PEFT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\pe_ch\anaconda3\envs\cairo-llm\Lib\site-packages\transformers\integrations\p

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pechaut/Mistral-C64Wizard-instruct-PEFT/commit/bcb85e18e31601805085364b65dbcd02520fae48', commit_message='Upload MistralForCausalLM', commit_description='', oid='bcb85e18e31601805085364b65dbcd02520fae48', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True,
                                             device_map={"": "cpu"},
                                             torch_dtype=torch.float16
                                             )

model_to_merge  = PeftModel.from_pretrained(model, new_model,
                        torch_dtype=torch.float16, 
                        device_map={"": "cpu"}
                         )
merged_model = model_to_merge.merge_and_unload()
#model.save_pretrained("cairo-mistral")
merged_model.push_to_hub(hub_name,max_shard_size="1GB")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/900M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00004-of-00015.safetensors:   0%|          | 0.00/956M [00:00<?, ?B/s]

...

model-00012-of-00015.safetensors:   0%|          | 0.00/956M [00:00<?, ?B/s]

model-00013-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pechaut/Mistral-C64Wizard-instruct/commit/d307acf721b4c2d18ffd1057be14bc76135f78df', commit_message='Upload MistralForCausalLM', commit_description='', oid='d307acf721b4c2d18ffd1057be14bc76135f78df', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [ ]:
quantization_config: 
load_in_8bit: False 
load_in_4bit: True 
llm_int8_threshold: 6.0 
llm_int8_skip_modules: None 
llm_int8_enable_fp32_cpu_offload: False 
llm_int8_has_fp16_weight: False 
bnb_4bit_quant_type: "nf4" 
bnb_4bit_use_double_quant: False 
bnb_4bit_compute_dtype: "float16"